### DATA CLEANING


In [73]:
import tensorflow as tf
import numpy as np
import pandas as pd

np.set_printoptions(precision=3, suppress=True)
data_train=pd.read_csv("train.csv")
data_train=data_train.sample(frac=1)
data_train=data_train.drop(['pickup_time','accept_time','allot_time','order_date','order_time','reassignment_reason','reassigned_order','rider_id','reassignment_method','cancelled_time','undelivered_orders','delivered_time'],axis=1)
labels=list(data_train.head(0))
x_id=data_train["order_id"]
y=data_train["cancelled"]
data_train=data_train.drop(["cancelled","order_id"],axis=1)

In [74]:
data_train.to_numpy()
data_train=np.nan_to_num(data_train)
data_train.shape


(450000, 6)

In [75]:
y.to_numpy()
train_examples=data_train
train_labels=y


In [76]:
data_test=pd.read_csv("test.csv")
data_test=data_test.drop(['accept_time','allot_time','order_date','order_time','reassignment_reason','reassigned_order','rider_id','reassignment_method','undelivered_orders'],axis=1)
labels=list(data_test.head(0))
test_id=data_test["order_id"]
y=data_test["cancelled"]
data_test=data_test.drop(["cancelled","order_id"],axis=1)
data_test.to_numpy()
data_test=np.nan_to_num(data_test)
y.to_numpy()
test_examples=data_test
test_labels=y

In [77]:
from sklearn.model_selection import train_test_split

# Let's say we want to split the data in 80:10:10 for train:valid:test dataset
train_size=0.8

X = train_examples
y = train_labels

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.6)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.3)


In [78]:
X_train.shape

(270000, 6)

In [85]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = int((X_train.shape[0])/1000)
tf.convert_to_tensor(X_train)

<tf.Tensor: shape=(270000, 6), dtype=float64, numpy=
array([[   3.28 ,    1.31 ,  270.   ,  269.   , 2214.   ,  342.567],
       [   1.651,    3.87 ,    1.   ,    1.   ,   16.   ,  156.95 ],
       [   2.257,    1.42 ,    4.   ,    4.   ,   78.   ,  164.483],
       ...,
       [   0.751,    4.68 ,   72.   ,   70.   , 3794.   ,  145.267],
       [   0.758,    5.37 ,    5.   ,    5.   ,   60.   ,  466.05 ],
       [   0.297,    1.11 ,   80.   ,   79.   , 7257.   ,    0.3  ]])>

In [36]:
'''normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X_train)'''

2022-02-02 14:22:15.545239: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 14:22:15.559457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [99]:
def get_basic_model():
  model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(6,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(6,activation= 'relu'),
    tf.keras.layers.Dense(3,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])
  opt = tf.keras.optimizers.SGD(lr=0.001)
  model.compile(optimizer=opt,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [94]:
X_train.shape

(270000, 6)

In [113]:
model = get_basic_model()
model.fit(X_train,y_train, epochs=1, batch_size=BATCH_SIZE,validation_data=(X_valid,y_valid))

/Users/aman/miniforge3/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/Users/aman/miniforge3/lib/python3.9/site-packages/keras/backend.py:4993: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(
2022-02-02 14:38:56.493441: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


 990/1000 [============================>.] - ETA: 0s - loss: 0.6342 - accuracy: 0.7806

/Users/aman/miniforge3/lib/python3.9/site-packages/keras/backend.py:4993: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(
2022-02-02 14:39:00.896843: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1000/1000 [==============================] - 6s 5ms/step - loss: 0.6327 - accuracy: 0.7826 - val_loss: 0.4784 - val_accuracy: 0.9806


In [114]:
model.evaluate(X_test,y_test)

1688/1688 [==============================] - 5s 3ms/step - loss: 0.4782 - accuracy: 0.9807


[0.47822853922843933, 0.9806666374206543]

In [115]:
tf.convert_to_tensor(test_examples)
ans=model.predict(test_examples)

2022-02-02 14:39:11.361087: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [119]:
max(ans)

array([0.836], dtype=float32)

In [121]:
for i, predicted in enumerate(ans):
    if predicted[0] >=0.5:
        print ("1")
        #assign i to class 0
ans.dtype

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


dtype('float32')

In [122]:
id=pd.Series([])
j=0
for i in test_id:
    id[j]=i
    j=j+1
len(id)

/var/folders/0_/jlz3w3sd0rvgpp49r0h93j_c0000gn/T/ipykernel_78627/3084436451.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  id=pd.Series([])


144844

In [123]:
data=pd.DataFrame()

In [124]:
id[0]

130231

In [125]:
data.insert(0,"order_id",id)

In [126]:
data.head()

,order_id
0,130231
1,130232
2,130233
3,130234
4,130235


In [127]:
scores=pd.Series([])
j=0
a=(sum(ans)/len(ans))
for i, predicted in enumerate(ans):
    if predicted[0] >= 0.5:
        scores[j]=1
    else:
        scores[j]=0
    j=j+1

/var/folders/0_/jlz3w3sd0rvgpp49r0h93j_c0000gn/T/ipykernel_78627/2878521483.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scores=pd.Series([])


In [128]:
scores[1]

0

In [129]:
data.insert(1,"cancelled",scores)

In [130]:
data.to_csv("model1_4.csv")

In [131]:
data.head()

,order_id,cancelled
0,130231,0
1,130232,0
2,130233,0
3,130234,0
4,130235,0
